In [14]:
# POWER METHOD

import numpy as np
from scipy.linalg import eigh

def run_GSE(rho, H, observable, m=1):
    def normalize_density_matrix(rho):
        """Normalize the density matrix to have a trace of 1 and non-negative eigenvalues."""
        trace_rho = np.trace(rho)
        if trace_rho == 0:
            raise ValueError("Trace of density matrix rho is zero, cannot normalize.")
        eigenvalues, eigenvectors = np.linalg.eigh(rho)
        eigenvalues_clipped = np.clip(eigenvalues, 0, None)
        return eigenvectors @ np.diag(eigenvalues_clipped) @ np.conj(eigenvectors.T)

    rho_normalized = normalize_density_matrix(rho)
    
    def is_valid_density_matrix(rho_normalized):
        """Check if rho matrix is a valid density matrix."""
        return np.allclose(np.trace(rho_normalized), 1) and np.all(np.linalg.eigvals(rho) >= 0)

    if not is_valid_density_matrix(rho_normalized):
        raise ValueError("The normalized rho is still not a valid density matrix.")
    
    def is_hermitian(matrix):
        """Check if a matrix is Hermitian."""
        return np.allclose(matrix, matrix.conj().T)
    
    if not is_hermitian(H_matrix):
        raise ValueError("H_matrix is not Hermitian.")


    A = rho_normalized if m % 2 != 0 else np.eye(rho_normalized.shape[0])
    powers_of_rho_normalized = [np.linalg.matrix_power(rho_normalized, i) for i in range(m + 1)]
    

    #H_MATRIX NEEDS TO BE PASSED!!!


    S_matrix = np.zeros((m+1, m+1), dtype=np.complex128)

    for i in range(m + 1):
        for j in range(m + 1):
            sigma_i = powers_of_rho_normalized[i].conj().T
            sigma_j = powers_of_rho_normalized[j]
            H_matrix[i, j] = np.trace(sigma_i @ A @ sigma_j @ H)
            S_matrix[i, j] = np.trace(sigma_i @ A @ sigma_j)

    eigenvalues, eigenvectors = eigh(H_matrix, S_matrix)
    min_eigenvalue_index = np.argmin(eigenvalues)
    alpha = eigenvectors[:, min_eigenvalue_index]
    rho_EM = sum(alpha[i] * powers_of_rho_normalized[i] for i in range(m + 1))
    expectation_value = np.real(np.trace(rho_EM @ observable))
    
    return expectation_value



In [16]:
#FAULT SUBSPACE METHOD

import numpy as np
from scipy.linalg import eigh

def run_GSE_fault_subspace(rho, H_matrix, observable, noise_levels, noise_function, stretch_factor=1.0, regularization_factor=1e-10):
    """
    Run the GSE using a fault subspace approach with a customizable noise function.

    Parameters:
    - rho: numpy.ndarray, the density matrix of the system.
    - H: numpy.ndarray, the system Hamiltonian.
    - observable: numpy.ndarray, the observable for which the expectation value is calculated.
    - noise_levels: list defining the noise levels or generating them dynamically.
    - noise_function: a function that applies noise to rho given noise levels.
    - stretch_factor: float, a factor to scale the noise levels, simulating stronger or weaker noise.
    
    Returns: 
    - float: the expectation value of the observable for the error-mitigated state.
    """
    
    def is_valid_starting_density_matrix(rho):
        """Check if a matrix is a valid density matrix."""
        return np.allclose(np.trace(rho), 1) and np.all(np.linalg.eigvals(rho) >= 0)

    if not is_valid_starting_density_matrix(rho):
        raise ValueError("initial rho is not a valid density matrix.")
    
    
    def normalize_density_matrix(rho):
        """Normalize the density matrix to have a trace of 1 and non-negative eigenvalues."""
        trace_rho = np.trace(rho)
        if trace_rho == 0:
            raise ValueError("Trace of density matrix is zero, cannot normalize.")
        eigenvalues, eigenvectors = np.linalg.eigh(rho)
        eigenvalues_clipped = np.clip(eigenvalues, 0, None)
        return eigenvectors @ np.diag(eigenvalues_clipped) @ np.conj(eigenvectors.T)

    rho_normalized = normalize_density_matrix(rho)
    
    def is_valid_density_matrix(rho_normalized):
        """Check if a matrix is a valid density matrix."""
        return np.allclose(np.trace(rho_normalized), 1) and np.all(np.linalg.eigvals(rho_normalized) >= 0)

    if not is_valid_density_matrix(rho_normalized):
        raise ValueError("The normalized rho is still not a valid density matrix.")
    

    # Apply the noise function at different levels and normalize each state
    noisy_states = [normalize_density_matrix(noise_function(rho_normalized, level * stretch_factor)) for level in noise_levels]



    # Check H_matrix Hermitian 
    def is_hermitian(matrix):
        """Check if a matrix is Hermitian."""
        return np.allclose(matrix, matrix.conj().T)
    if not is_hermitian(H_matrix):
        raise ValueError("Input H_matrix is not Hermitian.")
    
    # Initialize matrix_H with noise
    
    N = len(noisy_states)

    H_matrix_noisy = np.zeros((N, N), dtype=np.complex128)

    print("H_matrix_noisy shape:", H_matrix_noisy.shape)
    print("Shape of first noisy state:", noisy_states[0].shape)

    S_matrix_noisy = np.zeros((N, N), dtype=np.complex128)
    
    # Build the H and S matrices
    for i in range(N):
        for j in range(N):
            
            #assert noisy_states[i].shape == noisy_states[j].shape == H_matrix_noisy.shape, "Dimension mismatch"
            H_matrix_noisy[i, j] = np.trace(noisy_states[i].conj().T @ noisy_states[j] @ H_matrix)
            S_matrix_noisy[i, j] = np.trace(noisy_states[i].conj().T @ noisy_states[j])


    # Regularize S_matrix
    np.fill_diagonal(S_matrix_noisy, S_matrix_noisy.diagonal() + regularization_factor)

    # Solve the generalized eigenvalue problem
    eigenvalues, eigenvectors = eigh(H_matrix_noisy, S_matrix_noisy)
    min_eigenvalue_index = np.argmin(eigenvalues)
    optimal_alpha = eigenvectors[:, min_eigenvalue_index]

    rho_EM = sum(optimal_alpha[k] * noisy_states[k] for k in range(N))
    expectation_value = np.real(np.trace(rho_EM @ observable))
    
    return {
        "expectation_value": expectation_value,
        "error_mitigated_state": rho_EM,
        "eigenvalues": eigenvalues,
        "eigenvectors": eigenvectors,
        "diagnostics": {
            "trace_normalized_rho": np.trace(rho_normalized),
            "hermitian_check": is_hermitian(rho_EM)
        }
    }


In [17]:
# Example usage
def example_noise_model(rho, level):
    """Example noise model: Apply a simple depolarizing channel."""
    dim = rho.shape[0]
    identity = np.eye(dim) / dim
    return (1 - level) * rho + level * identity

rho = np.array([[0.7, 0.3], [0.3, 0.3]])  # Example density matrix, must be a valid quantum state
H_matrix = np.array([[1, 0], [0, 0]])  # Example Hamiltonian
observable = np.array([[1, 0], [0, 0]])  # Example observable
noise_levels = [0, 0.05, 0.1]  # Example noise levels

print('Error mitigated expectation value:', run_GSE_fault_subspace(rho, H_matrix, observable, noise_levels, example_noise_model))


H_matrix_noisy shape: (3, 3)
Shape of first noisy state: (2, 2)
Error mitigated expectation value: {'expectation_value': -3.092281986027956e-10, 'error_mitigated_state': array([[-3.09228199e-10+0.j,  7.69432518e-10+0.j],
       [ 7.67613528e-10+0.j, -1.36060407e-09+0.j]]), 'eigenvalues': array([-6.16496674e-07,  2.22649878e-01,  7.77350095e-01]), 'eigenvectors': array([[ 4.08248203e+04+0.j, -2.30341093e+01+0.j, -4.70083608e+00+0.j],
       [-8.16496406e+04+0.j,  3.33211214e-01+0.j, -3.33336331e-01+0.j],
       [ 4.08248203e+04+0.j,  2.37008980e+01+0.j,  4.03417238e+00+0.j]]), 'diagnostics': {'trace_normalized_rho': 1.0000000000000002, 'hermitian_check': True}}
